In [1]:
import os
os.chdir("../")

In [4]:
from langchain_ollama import OllamaLLM
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [20]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [2]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='data/')

In [6]:
extracted_data

[Document(metadata={'source': 'data\\1InvestmentandPortfolioManagement.pdf', 'page': 0}, page_content='PONDICHERRY UNIVERSITY\n(A Central University)\nDIRECTORATE OF DISTANCE EDUCATION\nInvestment and Portfolio Management\nMBA - FINANCE        Paper Code : MBFM 4001\nMBA - GENERAL        Paper Code : MBGN 4001\n  \nIV - Semester'),
 Document(metadata={'source': 'data\\1InvestmentandPortfolioManagement.pdf', 'page': 1}, page_content='Author\nDR. R. Kasilingam\nReader,\nDepartment of Management Studies,\nPondicherry University\nPuducherry\nAll Rights Reserved\nFor Private Circulation Only\nISBN 978-93-81932-13-1'),
 Document(metadata={'source': 'data\\1InvestmentandPortfolioManagement.pdf', 'page': 2}, page_content='             TABLE OF CONTENTS\nUNIT LESSON TITLE PAGE NO.\nI\n1.1  Investment 3\n1.2  Investment Alternatives 12\n1.3 Securities Market 27\n1.4 Stock Exchange 41\nII\n2.1 Fundamental Security Analysis - Economic Analysis 71\n2.2 Industry Analysis 81\n2.3 Company Analysis 91\

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 4512


In [9]:
text_chunks

[Document(metadata={'source': 'data\\1InvestmentandPortfolioManagement.pdf', 'page': 0}, page_content='PONDICHERRY UNIVERSITY\n(A Central University)\nDIRECTORATE OF DISTANCE EDUCATION\nInvestment and Portfolio Management\nMBA - FINANCE        Paper Code : MBFM 4001\nMBA - GENERAL        Paper Code : MBGN 4001\n  \nIV - Semester'),
 Document(metadata={'source': 'data\\1InvestmentandPortfolioManagement.pdf', 'page': 1}, page_content='Author\nDR. R. Kasilingam\nReader,\nDepartment of Management Studies,\nPondicherry University\nPuducherry\nAll Rights Reserved\nFor Private Circulation Only\nISBN 978-93-81932-13-1'),
 Document(metadata={'source': 'data\\1InvestmentandPortfolioManagement.pdf', 'page': 2}, page_content='TABLE OF CONTENTS\nUNIT LESSON TITLE PAGE NO.\nI\n1.1  Investment 3\n1.2  Investment Alternatives 12\n1.3 Securities Market 27\n1.4 Stock Exchange 41\nII\n2.1 Fundamental Security Analysis - Economic Analysis 71\n2.2 Industry Analysis 81\n2.3 Company Analysis 91\nIII\n3.1 Tec

In [10]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

c:\Users\sathv\anaconda3\envs\ragenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NOTE: embedding dimension = 384

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [12]:
vectordb = Chroma.from_documents(text_chunks, embedding=embeddings, persist_directory='./chroma')

In [13]:
# Load vector database from persisted directory
vectordb = Chroma(
    persist_directory="./chroma",
    embedding_function=embeddings
).as_retriever(search_type="similarity", k=3)

In [14]:
llm = OllamaLLM(model="hf.co/sathvik123/llama3-finnace-chat-gguf")

In [15]:
system_prompt = """
You are a financial advisor chatbot that provides clear, accurate, and helpful responses to financial questions. 
Explain financial concepts in simple terms, offer relevant examples when appropriate, and ensure your answers are concise, objective, and tailored to the user's query. 
Always use prior conversation history for context if the user's query is a follow-up question.
"""

# Define a custom prompt template
custom_prompt = PromptTemplate(
    input_variables=["chat_history", "question", "context"], 
    template=f"""
{system_prompt}

### Conversation History:
{{chat_history}}

### Retrieved Context:
{{context}}

### User Query:
{{question}}
"""
)

In [16]:
# Define memory to store conversation history
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

# Set up Conversational Retrieval Chain with memory
chatbot = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectordb,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt}
)

C:\Users\sathv\AppData\Local\Temp\ipykernel_13260\94923967.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


In [18]:
response = chatbot({"question": "Describe how the companies perform financial analysis."})
print("\nChatbot:", response["answer"])

C:\Users\sathv\AppData\Local\Temp\ipykernel_8236\1553904604.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chatbot({"question": "Describe how the companies perform financial analysis."})



Chatbot: Based on the provided context, we can assume that the user is asking about how companies conduct financial analysis. Here is a step-by-step explanation of the process:

1. Gathering Financial Statements: Companies obtain their financial statements, which include the income statement, balance sheet, and cash flow statement. These statements provide information about the company's operations, financial position, and cash flows.

2. Understanding the Statements: The next step is to thoroughly understand each financial statement. This involves analyzing the numbers and ratios presented in the statements to gain insights into the company's performance.

3. Comparing Historical Data: Companies compare their historical financial data with previous years' data to identify trends and patterns. This helps them assess whether their business is improving or deteriorating over time.

4. Analyzing Ratios: Financial analysts use various ratios, such as liquidity ratios (current ratio, quick

In [19]:
print(memory.buffer)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

Current summary:


New lines of conversation:
Human: Describe how companies perform financial analysis.
AI: Based on the provided context, we can assume that the user is asking about how companies conduct financial analysis. Here is a step-by-step explanation of the process:

1. Gathering Financial Statements: Companies obtain their financial statements, which include the income statement, balance sheet, and cash flow statement. These statements provide information about the company's operations, financial position, and cash flows.

2. Understanding the Statements: The next step is to thoroughly understand each financial statement. This involves analyzing the numbers and ratios presented in the statements to gain insights into the company's performance.

3. Comparing Historical Data: Companies compare their historical financial data with previous years' data to identif

In [20]:
response = chatbot({"question": "Describe its limitations"})
print("\nChatbot:", response["answer"])


Chatbot: Companies can overcome the limitations of financial analysis by taking the following steps:

1. Understanding the limitations: It is crucial to recognize that financial statement analysis has its limitations and biases. Economic influences, such as inflation or a recession, could skew the data being analyzed. Companies should be aware of these limitations and not solely rely on financial statements for decision-making.

2. Considering other sources of information: Financial statements are just one piece of information. Companies can consider additional sources such as market research, customer feedback, industry trends, and competitor analysis to gain a more comprehensive understanding of their business.

3. Analyzing qualitative factors: Qualitative factors like brand reputation, employee satisfaction, and management expertise should be considered alongside financial data. These factors can provide valuable insights into the company's performance and future prospects.

4. Co

In [21]:
print(memory.buffer)

Here is the updated summary:

The human asks how companies perform financial analysis. The AI explains the process step-by-step, including gathering financial statements, understanding the statements, comparing historical data, analyzing ratios, trend analysis, comparing with industry benchmarks, analyzing off-balance-sheet items, forecasting future performance, and reviewing by experts.

Companies can overcome the limitations of financial analysis by considering other sources of information, analyzing qualitative factors, conducting scenario planning, diversifying sources of funding, focusing on key performance indicators (KPIs), regularly updating and refining analysis, seeking expert opinions, conducting sensitivity analysis, and implementing internal controls and risk management practices.


In [22]:
response = chatbot({"question": "Describe different types"})
print("\nChatbot:", response["answer"])


Chatbot: Based on our previous conversation about fundamental analysis and its limitations, let's move on to discussing the different types of financial analysis. 

1. **Financial Statement Analysis**: This type of analysis involves examining a company's financial statements, such as balance sheet, income statement, and cash flow statement. It helps in understanding a company's performance by analyzing its profitability, solvency, liquidity, and cash flows.

2. **Ratio Analysis**: Ratio analysis is a subcategory of financial statement analysis that involves calculating various financial ratios to evaluate a company's performance. These ratios can be classified into different categories such as liquidity ratios, profitability ratios, efficiency ratios, and solvency ratios. Ratios help in identifying trends, comparing companies within an industry, and evaluating a company's performance over time.

3. **Trend Analysis**: Trend analysis involves analyzing the patterns and trends of a comp

In [23]:
print(memory.buffer)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

Current summary:
The human asks how companies perform financial analysis. The AI explains the process step-by-step, including gathering financial statements, understanding the statements, comparing historical data, analyzing ratios, trend analysis, comparing with industry benchmarks, analyzing off-balance-sheet items, forecasting future performance, and reviewing by experts.

Companies can overcome the limitations of financial analysis by considering other sources of information, analyzing qualitative factors, conducting scenario planning, diversifying sources of funding, focusing on key performance indicators (KPIs), regularly updating and refining analysis, seeking expert opinions, conducting sensitivity analysis, and implementing internal controls and risk management practices.

New lines of conversation:
Human: Describe different types
AI: Based on our previous con